In [ ]:
import pandas as pd
import joblib

In [ ]:
TARGET = "NO2"
OSM_ID = 8269826
MAP_HEX_SIZE = 7
SELECTED_YEARS = (2023, 2024)
years_str = "_".join(str(year) for year in SELECTED_YEARS)
MODEL_COMMENT = "artificial_S5P_scaled_wind_shift" # String or None
DATA_COMMENT = "artificial_S5P_scaled_wind_shift" # String or None
if DATA_COMMENT:
    ML_INFERENCE_DATA_FILE = f"../data/{TARGET}_inference_dataset_osm_{OSM_ID}_hex_{MAP_HEX_SIZE}_year_{years_str}_{DATA_COMMENT}.csv"
else:
    ML_INFERENCE_DATA_FILE = f"../data/{TARGET}_inference_dataset_osm_{OSM_ID}_hex_{MAP_HEX_SIZE}_year_{years_str}.csv"
if MODEL_COMMENT:
    ML_MODEL = f"../data/random_forest_{TARGET}_gios_{MODEL_COMMENT}.pkl"
else:
    ML_MODEL = f"../data/random_forest_{TARGET}_gios.pkl"
SELECTED_PARAMETERS = [
    "tree_cover",
    "grassland",
    "population_density",
    "low_vegetation",
    "medium_vegetation",
    "high_vegetation",
    "road",
    "residential_1",
    "residential_2",
    "residential_3",
    "residential_4",
    "non-residential_1",
    "non-residential_2",
    "non-residential_3",
    "non-residential_4",
    "temperature",
    "temperature_trend_3h",
    "temperature_trend_6h",
    "temperature_anomaly",
    "relative_humidity",
    "relative_humidity_trend_3h",
    "relative_humidity_trend_6h",
    "pressure",
    "pressure_trend_3h",
    "pressure_trend_6h",
    "precipitation",
    "wind_speed",
    "traffic_mean_count",
    "day_of_year_sin",
    "day_of_year_cos",
    "working_day",
]

In [ ]:
df_inference = pd.read_csv(ML_INFERENCE_DATA_FILE)
df_inference = df_inference[["timestamp_utc", "h3_index"] + SELECTED_PARAMETERS]
df_inference.dropna(inplace=True)
df_inference["timestamp_utc"] = pd.to_datetime(df_inference["timestamp_utc"])
df_inference.head()

In [ ]:
df_output = df_inference[["timestamp_utc", "h3_index"]].copy()
df_inference = df_inference.drop(columns=["timestamp_utc", "h3_index"]).copy()

In [ ]:
rf_model = joblib.load(ML_MODEL)

In [ ]:
df_output[f"{TARGET}_inference"] = rf_model.predict(df_inference)
df_output.head()

In [ ]:
df_shp = df_output.copy()

In [ ]:
import geopandas as gpd
import h3
from shapely.geometry import Polygon

In [ ]:
def cell_to_polygon(cell):
    # Coordinates in (latitude, longitude) format
    coords = h3.cell_to_boundary(cell)
    # Switch to (longitude, latitude) format
    flipped_coords = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped_coords)

In [ ]:
df_shp["timestamp_utc"] = pd.to_datetime(df_shp["timestamp_utc"])
df_shp["timestamp_utc"] = df_shp["timestamp_utc"].dt.strftime("%m-%dT%H")

In [ ]:
df_shp_pivot = df_shp.pivot_table(
    index=["h3_index"],
    columns="timestamp_utc",
    values="NO2_inference",
)

df_shp_pivot = df_shp_pivot.reset_index()
df_shp_pivot.head()

In [ ]:
df_shp_pivot["geometry"] = df_shp_pivot.apply(lambda row: cell_to_polygon(row["h3_index"]), axis=1)
df_shp_pivot.head()

In [ ]:
gdf_shp_pivot = gpd.GeoDataFrame(df_shp_pivot, geometry="geometry")
gdf_shp_pivot = gdf_shp_pivot.set_crs(crs="EPSG:4326")

In [ ]:
if COMMENT:
    gdf_shp_pivot.to_file(f"inference_NO2_osm_{OSM_ID}_hex_{MAP_HEX_SIZE}_{COMMENT}.shp", driver="ESRI Shapefile")
else:
    gdf_shp_pivot.to_file(f"inference_NO2_osm_{OSM_ID}_hex_{MAP_HEX_SIZE}.shp", driver="ESRI Shapefile")